# Semi Supervised GAN for image classification

Pytorch implementation

## The Data

Uses [DALI by NVIDIA](https://github.com/NVIDIA/DALI) to speedup the pre-processing (namely, resizing) of the images

In [ ]:
# !pip install --extra-index-url https://developer.download.nvidia.com/compute/redist --upgrade nvidia-dali-cuda110


In [ ]:
!pip install --extra-index-url https://developer.download.nvidia.com/compute/redist/cuda/11.0 nvidia-dali==0.22.0


In [ ]:
from __future__ import print_function, division

import random
from random import shuffle

import numpy as np
import matplotlib.pyplot as plt
import copy

import os
from os import listdir
from os.path import join

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torch.nn.functional as F

import torchvision
import torchvision.utils as vutils
from torchvision import utils, models

from nvidia.dali.pipeline import Pipeline
import nvidia.dali.ops as ops
import nvidia.dali.types as types
from nvidia.dali.plugin.pytorch import DALIGenericIterator, DALIClassificationIterator

import warnings
warnings.filterwarnings("ignore")

import os
# len(os.listdir('./data/train'))

import pandas as pd

from PIL import Image
from zipfile import ZipFile

import shutil

In [ ]:
zip_path_train = '/kaggle/input/dogs-vs-cats/train.zip'
with ZipFile(zip_path_train) as myzip:
    files_in_zip_train = myzip.namelist()

In [ ]:
zip_path_train = '/kaggle/input/dogs-vs-cats/test1.zip'
with ZipFile(zip_path_train) as myzip:
    files_in_zip_test = myzip.namelist()

In [ ]:
# extracting all files
base_dir = "/kaggle/input/dogs-vs-cats/"
train_dir = os.path.join(base_dir, "train.zip")
test_dir = os.path.join(base_dir, "test1.zip")

import zipfile
with zipfile.ZipFile(train_dir,"r") as z:
    z.extractall()

with zipfile.ZipFile(test_dir,"r") as z:
    z.extractall()

In [ ]:
os.makedirs('./data/labelled')
os.makedirs('./data/train')
os.makedirs('./data/test')
os.makedirs('./data/validate')


In [ ]:
#labelled dataset -> 100 images

#cats
for filename in files_in_zip_train[1:51]:
       shutil.move(os.path.join(filename),'./data/labelled')

#dogs
for filename in files_in_zip_train[-50:]:
       shutil.move(os.path.join(filename),'./data/labelled')

In [ ]:
#validate set 1000 from labelled   
for filename in files_in_zip_train[51:551]:
       shutil.move(os.path.join(filename),'./data/validate')
        
for filename in files_in_zip_train[-551:-51]:
       shutil.move(os.path.join(filename),'./data/validate')

In [ ]:
#test set 1000 from labelled    
for filename in files_in_zip_train[551:1051]:
       shutil.move(os.path.join(filename),'./data/test')
        
for filename in files_in_zip_train[-1051:-551]:
       shutil.move(os.path.join(filename),'./data/test')

In [ ]:
#train -> remaining labelled and unlabelled data
for filename in files_in_zip_train[1051:-1051]:
       shutil.move(os.path.join(filename),'./data/train')
        
for filename in files_in_zip_test[1:]:
       shutil.move(os.path.join(filename),'./data/train')

In [ ]:
len(os.listdir('./data/train'))

In [ ]:
len(os.listdir('./data/labelled'))

In [ ]:
len(os.listdir('./data/test'))


In [ ]:
len(os.listdir('./data/validate'))


In [ ]:
np.random.seed(42)
random.seed(10)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.manual_seed(999)

device = torch.device("cuda")

labeled_train_dir = './data/labelled'
labeled_batch = 100 # labeled_batch = total number of labeled training samples since it is small enough
unlabeled_train_dir = './data/train'
val_dir = './data/validate'
test_dir = './data/test'

def is_jpeg(filename):
    return any(filename.endswith(extension) for extension in [".jpg", ".jpeg"])

class ExternalInputIterator(object):

    def __init__(self, imageset_dir, labeled, batch_size, random_shuffle=False):
        self.images_dir = imageset_dir
        self.batch_size = batch_size
        self.labeled = labeled # True is images are labeled, False otherwise

        self.image_files = np.array([join(imageset_dir, file_name) for file_name in sorted(listdir(imageset_dir)) if is_jpeg(file_name)])
        
        # In my dataset, filenames are of the form "cat_***.jpg" or "*cat_***.jpg" (same for dogs)
        # The map below infers the image's label from its the second character of its filename
        self.class_map = {'a': 0, 'o': 1, 'c': 0, 'd': 1}
        self.offset = len(self.images_dir) + 2
        
        if random_shuffle:
            ind = np.array(range(len(self.image_files)))
            shuffle(self.image_files)

    def __iter__(self):
        self.i = 0
        self.n = len(self.image_files)
        return self


    # Return a batch of (input, target) pairs
    def __next__(self):
        
        images = []
        labels = []

        for _ in range(self.batch_size):
            image_file = self.image_files[self.i]
            if self.labeled:
                label = self.class_map[self.image_files[self.i][self.offset]]
                labels.append(np.array([label], dtype = np.uint8))
            image = open(image_file, 'rb')
            images.append(np.frombuffer(image.read(), dtype = np.uint8))
            image.close()

            self.i = (self.i + 1) % self.n
        if self.labeled:
            return (images, labels)
        return images

    next = __next__

class PetPipeline(Pipeline):
    '''
    Constructor arguments:  
    - imageset_dir: directory containing the dataset
    - labeled = True
    - image_size = 128: length of the square that the images will be resized to
    - random_shuffle = False
    - batch_size = 64
    - num_threads = 1
    - device_id = 0
    '''

    def __init__(self, imageset_dir, labeled=True, image_size=128, random_shuffle=False, batch_size=64, num_threads=1, device_id=0):
        super(PetPipeline, self).__init__(batch_size, num_threads, device_id, seed=12)  
        eii = ExternalInputIterator(imageset_dir, labeled, batch_size, random_shuffle)
        self.iterator = iter(eii)
        self.num_inputs = len(eii.image_files)
        self.labeled = labeled

        # The source for the inputs
        self.input = ops.ExternalSource()
        self.input_label = ops.ExternalSource()

        self.decode = ops.ImageDecoder(device = 'mixed', output_type = types.RGB)

        # The rest of pre-processing is done on the GPU
        self.res = ops.Resize(device="gpu", resize_x=image_size, resize_y=image_size)
        self.norm = ops.CropMirrorNormalize(device='gpu', mean=0, std=255.0)


    # epoch_size = number of (profile, frontal) image pairs in the dataset
    def epoch_size(self, name = None):
        return self.num_inputs


    # Define the flow of the data loading and pre-processing
    def define_graph(self):   
        
        if self.labeled:
            self.labels = self.input_label()
        self.jpegs = self.input()
        images = self.decode(self.jpegs)
        images = self.res(images)
        output = self.norm(images)
        if self.labeled:
            self.labels = self.input_label()
            return (output, self.labels)
        else:
            return output
    
    def iter_setup(self):
        if self.labeled:
            (images, labels) = self.iterator.next()
            self.feed_input(self.jpegs, images)
            self.feed_input(self.labels, labels)
        else:
            images = self.iterator.next()
            self.feed_input(self.jpegs, images)
    
# Test labeled and unlabeled dataloaders:
    
data_pipe = PetPipeline(labeled_train_dir, True, image_size=224, random_shuffle=True, batch_size=16)
data_pipe.build()
# Use DALIClassificationIterator for the labeled Pipeline
data_loader = DALIClassificationIterator(data_pipe, data_pipe.epoch_size())

test_batch = next(iter(data_loader))
images_batch = test_batch[0]['data'].cpu()
labels_batch = test_batch[0]['label']
fig = plt.figure(figsize=(20,10))
grid = utils.make_grid(images_batch)
plt.imshow(grid.numpy().transpose((1, 2, 0)))
plt.show()
print('Labels: ', labels_batch.T)

data_pipe = PetPipeline(labeled_train_dir, False, image_size=224, random_shuffle=True, batch_size=16)
data_pipe.build()
# Use DALIGenericIterator for the unlabeled Pipeline
data_loader = DALIGenericIterator(data_pipe, ['data'], data_pipe.epoch_size())

test_batch = next(iter(data_loader))
images_batch = test_batch[0]['data'].cpu()
fig = plt.figure(figsize=(20,10))
grid = utils.make_grid(images_batch)
plt.imshow(grid.numpy().transpose((1, 2, 0)))
plt.show()

# Supervised ResNet18 trained on 100 training samples

In [ ]:
# test_pipe = PetPipeline(labeled_train_dir, True, image_size=224, random_shuffle=True, batch_size=512)
test_pipe = PetPipeline(test_dir, True, image_size=224, random_shuffle=True, batch_size=100)

test_pipe.build()

m_test = test_pipe.epoch_size()

test_loader = DALIClassificationIterator(test_pipe, m_test)

In [ ]:
# final_test_pipe = PetPipeline(test_dir, True, image_size=224, random_shuffle=True, batch_size=100)
# final_test_pipe.build()
# final_test_loader = DALIClassificationIterator(final_test_pipe, final_test_pipe.epoch_size())

labeled_pipe = PetPipeline(labeled_train_dir, True, image_size=224, random_shuffle=False, batch_size=labeled_batch)
labeled_pipe.build()
m_l = labeled_pipe.epoch_size()
labeled_loader = DALIClassificationIterator(labeled_pipe, m_l)

labeled_batch = next(iter(labeled_loader))
labeled_data = labeled_batch[0]['data'].type('torch.FloatTensor').to(device)
labels = labeled_batch[0]['label'].type(dtype=torch.long).squeeze().to(device)

def test(model, device, test_loader, m_test, display = False):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for batch in test_loader:
            data = batch[0]['data'].type('torch.FloatTensor').to(device)
            target = batch[0]['label'].type(dtype=torch.long).squeeze().to(device)
            output = model(data)
            test_loss += criterion(output, target).item() 
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= m_test

    if display == True:
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, m_test,
        100. * correct / m_test))
        
    test_loader.reset()    
    return test_loss, 100. * correct / m_test


epochs = 100
lr = 0.1
gamma = 0.1

# Use a (not pre-trained) ResNet18 with d_out = 2
classifier = models.resnet18()
num_ftrs = classifier.fc.in_features
classifier.fc = nn.Linear(num_ftrs, 2)
classifier = classifier.to(device)

optimizerC = optim.SGD(classifier.parameters(), lr=0.001, momentum=0.9, dampening=0, weight_decay=0.0001)

criterion = nn.CrossEntropyLoss()

epochs = 200

test_losses = []
test_accuracies = []

best_model_wts = copy.deepcopy(classifier.state_dict())
best_acc = 0.0

for epoch in range(1, epochs + 1):
    
    classifier.train()
    
    optimizerC.zero_grad()
    output = classifier(labeled_data)
    loss = criterion(output, labels)
    loss.backward()
    
    if epoch % 50 == 0:
        print('Train Epoch: {} \tLoss: {:.6f}'.format(epoch, loss.item()))
    
        test_loss, test_accuracy = test(classifier, device, test_loader, m_test, True)
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)
    else:
        test_loss, test_accuracy = test(classifier, device, test_loader, m_test, False)
        test_losses.append(test_loss)
        test_accuracies.append(test_accuracy)
        
    if test_accuracy > best_acc:
        best_acc = test_accuracy
        best_classifier_wts = copy.deepcopy(classifier.state_dict())
        
    optimizerC.step()
        
plt.plot(range(len(test_losses)), np.array(test_losses))
plt.show()

print('Best VAL test accuracy: ', np.max(np.array(test_accuracies)),
      '% after ', np.argmax(np.array(test_accuracies)), ' training epochs')

# load best model weights
classifier.load_state_dict(best_classifier_wts)

print("\n Best model on the test set: ")
# test(classifier, device, final_test_loader, final_test_pipe.epoch_size(), True)
test(classifier, device, test_loader, test_pipe.epoch_size(), True)

# Semi supervised GAN

* Generator + ResNet18 classifier used as a Discriminator
* 100 labeled training samples
* Around 30K unlabeled images

In [ ]:
batch_size = 64

epochs = 14

train_pipe = PetPipeline(unlabeled_train_dir, False, image_size=224, random_shuffle=True, batch_size=batch_size)
train_pipe.build()
m_train = train_pipe.epoch_size()
train_loader = DALIGenericIterator(train_pipe, ['data'], m_train)

def log_sum_exp(x, axis = 1):
    m = torch.max(x, dim = 1)[0]
    return m + torch.log(torch.sum(torch.exp(x - m.unsqueeze(1)), dim = axis))

# custom weights initialization called on the Generator
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)
        

# Number of channels in the training images. For color images this is 3
nc = 3
# Size of z latent vector 
nz = 100
# Size of feature maps in generator
ngf = 64

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # input is Z, going into a convolution
            nn.ConvTranspose2d( nz, ngf * 8, 7, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. (ngf*8) x 7 x 7
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. (ngf*4) x 14 x 14
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. (ngf*2) x 28 x 28
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 4, 0, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. (ngf) x 112 x 112
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. (nc) x 224 x 224
        )

    def forward(self, input):
        return self.main(input)
    
# A noise vector to be used for generating images at the end of each training epoch
fixed_noise = torch.randn(64, nz, 1, 1, device=device)

netG = Generator().to(device)
netG.apply(weights_init)

classifier = models.resnet18()
num_ftrs = classifier.fc.in_features
classifier.fc = nn.Linear(num_ftrs, 2)
classifier = classifier.to(device)

optimizerG = optim.Adam(netG.parameters(), lr=0.002, betas= (0.5, 0.999))
optimizerC = optim.Adam(classifier.parameters(), lr=0.002, betas= (0.5, 0.999))#, dampening=0, weight_decay=0.0001)

test_losses = []
test_accuracies = []

best_model_wts = copy.deepcopy(classifier.state_dict())
best_acc = 0.0

for epoch in range(1, epochs + 1):

    for batch_idx, batch in enumerate(train_loader):
        
        # TRAIN THE DISCRIMINATOR (THE CLASSIFIER)
        classifier.train()
        optimizerC.zero_grad()
        
        # 1. on Unlabelled data
        data = batch[0]['data'].type('torch.FloatTensor').to(device)        
        outputs = classifier(data)    
        logz_unlabel = log_sum_exp(outputs)
        lossUL = 0.5 * (-torch.mean(logz_unlabel) + torch.mean(F.softplus(logz_unlabel)))
        lossUL.backward()  
        
        # 2. on the generated data

        noise = torch.randn(batch_size, 100, 1, 1, device=device)
        generated = (netG(noise)+1.0)/2.0
        outputs = classifier(generated.detach()) # detach() because we are not training G here
        logz_fake = log_sum_exp(outputs)
        lossD = 0.5*torch.mean(F.softplus(logz_fake))
        lossD.backward()
        
        # 3. on labeled data
        output = classifier(labeled_data)
        logz_label = log_sum_exp(output)
        prob_label = torch.gather(output, 1, labels.unsqueeze(1))
        labeled_loss = -torch.mean(prob_label) + torch.mean(logz_label)
        labeled_loss.backward()    

        optimizerC.step()
        
        # TRAIN THE DISCRIMINATOR (THE CLASSIFIER)
        netG.train()
        optimizerG.zero_grad()
        
        outputs = classifier(generated)
        logz_unlabel = log_sum_exp(outputs)
        lossG = 0.5 * (-torch.mean(logz_unlabel) + torch.mean(F.softplus(logz_unlabel)))
        lossG.backward()
        optimizerG.step()
            
    train_loader.reset()
    
    generated = (netG(fixed_noise)+1.0)/2.0
    vutils.save_image(generated.cpu().detach(), ('generated_%d.jpg' % epoch), normalize=True)
    
    test_loss, test_accuracy = test(classifier, device, test_loader, m_test, True)
    test_losses.append(test_loss)
    test_accuracies.append(test_accuracy)
        
    if test_accuracy > best_acc:
        best_acc = test_accuracy
        best_classifier_wts = copy.deepcopy(classifier.state_dict())

plt.plot(range(len(test_losses)), np.array(test_losses))
plt.show()

print('Best VAL test accuracy: ', np.max(np.array(test_accuracies)),
      '% after ', np.argmax(np.array(test_accuracies)), ' training epochs')

# load best model weights
classifier.load_state_dict(best_classifier_wts)

print("\nBest model on the test set: ")
# test(classifier, device, final_test_loader, final_test_pipe.epoch_size(), True)
test(classifier, device, test_loader, test_pipe.epoch_size(), True)